1.AGENTE QUE USA FERRAMENTAS DE PESQUISA NA WEB

IMPORTANDO AS BIBLIOTECAS NECESSÁRIAS

In [ ]:
from crewai_tools import ScrapeWebsiteTool, FileWriterTool, TXTSearchTool
from crewai import Agent, Task, Crew
import os
from dotenv import load_dotenv


load_dotenv()
#Definindo a chave de API do OpenAI
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

REALIZANDO O SCRAPE DE UM SITE

In [ ]:
#Definição de uma ferramenta de "raspagem" de texto de um site
tool = ScrapeWebsiteTool(website_url='https://en.wikipedia.org/wiki/Artificial_intelligence')  

#Executando a ferramenta e armazenando o texto raspado
text = tool.run()
print(text)

SALVANDO O QUE FOI EXTRAIDO EM UM ARQUIVO TXT

In [ ]:
#Definindo uma ferramenta de escrita de arquivo
file_writer_tool = FileWriterTool()
text = text.encode("ascii", "ignore").decode()

#Executando a ferramenta de escrita de arquivo  e armazenando o resultado
result = file_writer_tool._run(filename='ai.txt', content = text, overwrite="True")
print(result)

Content successfully written to ai.txt


CONFIGURAÇÃO DA FERRAMENTA DE PESQUISA NO ARQUIVO DE TEXTO

In [ ]:
#Definindo uma ferramenta de busca de texto em um arquivo
tool = TXTSearchTool(txt='ai.txt')

Inserting batches in chromadb: 100%|██████████| 2/2 [00:09<00:00,  4.76s/it]


In [ ]:
#Executando a ferramenta de busca de texto em um arquivo
#com a pergunta "O que é processamento de linguagem natural?"
context = tool.run('What is natural language processing?')

Using Tool: Search a txt's content


CRIAÇÃO DO AGENTE

In [ ]:
#Criação do Agente por meio de uma instanciação da classe Agent
data_analyst = Agent(
    #Aqui é definido o nome/função do agente
    role='Educator',
    #Aqui é definido o objetivo do agente
    goal=f'Based on the context provided, answer the question - What is Natural Language Processing? Context - {context}',
    #Aqui é definida a "identidade" do agente,
    #ou seja, o modo como ele irá agir
    backstory='You are a data expert',
    #Parâmetro que define se as mensagens sobre as 
    #ações realizadas pelos agentes serão exibidas
    verbose=True,
    #Aqui é definido que ele não pode delegar tarefas
    allow_delegation=False,
    #Aqui é definida as ferramentas que o agente pode usar
    tools=[tool]
)

CRIAÇÃO DA TASK

In [ ]:
#Criação de uma tarefa por meio de uma instanciação da classe Task
test_task = Task(
    #Aqui é definida a descrição da tarefa, o que esperamos que o agente faça, 
    #a tarefa planejada, nesse caso, é entender o tópico e dar a resposta correta
    description="Understand the topic and give the correct response",
    #Aqui é definida a ferramenta que será usada para realizar a tarefa
    tools=[tool],
    #Aqui é definido o agente que irá realizar a tarefa
    agent=data_analyst,
    #Aqui é definido o que é esperado como saída da tarefa
    expected_output='Give a correct response'
)


CRIAÇÃO DO CREW

In [ ]:
crew = Crew(
    #Definição dos agentes que compõem a equipe
    agents=[data_analyst],
    #Definição das tarefas que a equipe deve realizar
    tasks=[test_task]
)

INICIALIZANDO PARA VER SE FUNCIONOU

In [16]:
output = crew.kickoff()
print(output)

# Agent: Educator
## Task: Understand the topic and give the correct response


# Agent: Educator
## Using tool: Search a txt's content
## Tool Input: 
"{\"search_query\": \"Natural Language Processing\"}"
## Tool Output: 
Relevant Content:
Hughes-Castleberry, Kenna, "A Murder Mystery Puzzle: The literary puzzle Cain's Jawbone , which has stumped humans for decades, reveals the limitations of natural-language-processing algorithms", Scientific American , vol. 329, no. 4 (November 2023), pp.8182. "This murder mystery competition has revealed that although NLP ( natural-language processing ) models are capable of incredible feats, their abilities are very much limited by the amount of context they receive. This [...] could cause [difficulties] for researchers who hope to use them to do things such as analyze ancient languages . In some cases, there are few historical records on long-gone civilizations to serve as training data for such a purpose." (p.82.)
Immerwahr, Daniel , "Your Lying 

2. AGENTE DE CAMPANHA DE RECOMENDAÇÃO

IMPORTANDO BIBLIOTECAS NECESSÁRIAS

In [ ]:
from crewai import Agent, Task, Crew, Process
from textwrap import dedent
import pandas as pd

import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()
#Definindo a chave de API do OpenAI
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

DEFINIÇÃO DO MODELO A SER USADO

In [ ]:
#Definição do modelo de linguagem a ser utilizado
llm = ChatOpenAI(
    model="gpt-3.5-turbo-16k",
    temperature=0.1,
    max_tokens=8000
)

In [ ]:
#Arquivo csv que contém informações sobre os alunos
csv = '''Academic Goals, Major, Hobbies, Computer Skills, Interest in Languages, GPA
To become a software engineer, Computer Science, Gaming, Advanced, Spanish, 3.7
To study environmental science, Environmental Science, Hiking, Intermediate, French, 3.5
To pursue a career in medicine, Pre-Med, Playing the piano, Advanced, Spanish, 3.9
To major in psychology, Psychology, Reading, Intermediate, German, 3.6
To work in international relations, Political Science, Traveling, Basic, Mandarin, 3.8
To become a teacher, Education, Painting, Advanced, Spanish, 3.4
To study literature, English Literature, Writing, Intermediate, French, 3.9
To pursue a career in business, Business Administration, Playing soccer, Basic, Mandarin, 3.5
To become a biologist, Biology, Photography, Advanced, German, 3.7
To work in data analysis, Statistics, Cooking, Intermediate, Japanese, 3.6
'''

from io import StringIO
csvStringIO = StringIO(csv)
df_customers = pd.read_csv(csvStringIO, sep=",")

#Definição dos cursos disponíveis e suas informações
courses = '''
"Introduction to Computer Science" - Offered by Harvard University on edX
"Biology: Life on Earth" - Offered by Coursera
"Introduction to Psychology" - Offered by Yale University on Coursera
"Environmental Science" - Offered by University of Leeds on FutureLearn
"Introduction to Literature" - Offered by MIT on edX
"Medical Terminology" - Offered by University of Pittsburgh on Coursera
"Data Science and Machine Learning" - Offered by Stanford University on Coursera
"Cell Biology" - Offered by Massachusetts Institute of Technology on edX
"Positive Psychology" - Offered by University of North Carolina at Chapel Hill on Coursera
"Environmental Law and Policy" - Offered by Vermont Law School on Coursera
"Programming for Everybody (Getting Started with Python)" - Offered by University of Michigan on Coursera
"Anatomy: Human Neuroanatomy" - Offered by University of Michigan on Coursera
"Introduction to Cognitive Psychology" - Offered by Duke University on Coursera
"Climate Change and Health: From Science to Action" - Offered by Harvard University on edX
"English for Science, Technology, Engineering, and Mathematics" - Offered by University of Pennsylvania on Coursera
"An Introduction to American Law" - Offered by University of Pennsylvania on Coursera
"Introduction to Chemistry: Reactions and Ratios" - Offered by Duke University on Coursera
"Epidemiology: The Basic Science of Public Health" - Offered by University of North Carolina at Chapel Hill on Coursera
"Computer Science: Programming with a Purpose" - Offered by Princeton University on Coursera
"Introduction to Statistics and Data Analysis" - Offered by Rice University on Coursera
"Genes and the Human Condition (From Behavior to Biotechnology)" - Offered by University of Maryland on Coursera
"Ethics, Technology, and the Future of Medicine" - Offered by Georgetown University on edX
"Fundamentals of Immunology" - Offered by Harvard University
'''

DEFINIÇÃO DOS AGENTES

In [ ]:
#Criação do Agente por meio de uma instanciação da classe Agent
student_profiler = Agent(
  #Aqui é definido o nome/função do agente
  role='student_profiler',
  #Aqui é definido o objetivo do agente
  goal='''From limited data, you logically deduct conclusions about students.''',
  #Aqui é definida a "identidade" do agente,
  #ou seja, o modo como ele irá agir
  backstory='You are an expert psychologist with decades of experience.',
  #Definição do modelo de linguagem a ser utilizado
  llm = llm,
  #Aqui é definido que ele não pode delegar tarefas
  allow_delegation=False,
  #Parâmetro que define se as mensagens sobre as 
  #ações realizadas pelos agentes serão exibidas
  verbose=True)


In [ ]:
#Criação do Agente por meio de uma instanciação da classe Agent
course_specialist = Agent(
     #Aqui é definido o nome/função do agente
     role='course specialist',
     #Aqui é definido o objetivo do agente
     goal='''Match the suitable course to the students''',
     #Aqui é definida a "identidade" do agente,
     #ou seja, o modo como ele irá agir
     backstory='You have exceptional knowledge of the courses and can say how valuable they are to a student.',
     #Definição do modelo de linguagem a ser utilizado
     llm = llm,
     #Aqui é definido que ele não pode delegar tarefas
     allow_delegation=False,
     #Parâmetro que define se as mensagens sobre as
     #ações realizadas pelos agentes serão exibidas
     verbose=True)


In [ ]:
#Criação de uma tarefa por meio de uma instanciação da classe Task
Chief_Recommendation_Director = Agent(
   #Aqui é definido o nome/função do agente
   role="Chief Recomeendation Director",
   #Aqui é definido o objetivo do agente
   goal=dedent("""\Oversee the work done by your team to make sure it's the best
possible and aligned with the course's goals, review, approve,
ask clarifying question or delegate follow up work if necessary to make
decisions"""),
   #Aqui é definida a "identidade" do agente,
   #ou seja, o modo como ele irá agir 
   backstory=dedent("""\You're the Chief Promotion Officer of a large EDtech company. You're launching a personalized ad campaign,
trying to make sure your team is crafting the best possible content for the customer."""),
   #Definição do modelo de linguagem a ser utilizado
     llm = llm,
     #Definição das ferramentas que o agente pode usar, ou seja, nenhuma
     tools=[],
     #Aqui é definido que ele não pode delegar tarefas
     allow_delegation=False,
     #Parâmetro que define se as mensagens sobre as
     #ações realizadas pelos agentes serão exibidas
     verbose=True)

In [ ]:
campaign_agent = Agent(
    #Aqui é definido o nome/função do agente
     role="campaign_agent",
     #Aqui é definido o objetivo do agente
     goal=dedent("""\Develop compelling and innovative content
  for ad campaigns, with a focus customer specific ad copies."""),
     #Aqui é definida a "identidade" do agente,
     # ou seja, o modo como ele irá agir    
     backstory=dedent("""\As a Creative Content Creator at a top-tier
   digital marketing agency, you excel in crafting advertisements
   that resonate with potential customers.
   Your expertise lies in turning marketing strategies
   into engaging stories that capture
   attention and inspire buying action."""),
     #Definição do modelo de linguagem a ser utilizado
     llm = llm,
     #Aqui é definido que ele não pode delegar tarefas
     allow_delegation=False,
     #Parâmetro que define se as mensagens sobre as
     #ações realizadas pelos agentes serão exibidas
     verbose=True)

DEFINIÇÃO DAS TASKS

In [ ]:
def get_ad_campaign_task(agent, customer_description, courses):
  #Criação de uma tarefa por meio de uma instanciação da classe Task
  return Task(
    #Definição da descrição da tarefa
    description=dedent(f"""\
    You're creating a targeted marketing campaign tailored to what we know about our student customers.

    For each student customer, we have to choose exactly three courses to promote in the next campaign.
    Make sure the selection is the best possible and aligned with the student customer,
   review, approve, ask clarifying question or delegate follow up work if
  necessary to make decisions. When delegating work send the full draft
  as part of the information.
    This is the list of all the courses participating in the campaign: {courses}.
    This is all we know so far from the student customer: {customer_description}.

    To start this campaign we will need to build first an understanding of our student customer.
    Once we have a profile about the student customers interests, lifestyle and means and needs,
    we have to select exactly three courses that have the highest chance to be bought by them.

    Your final answer MUST be exactly 3 courses from the list, each with a short description
    why it matches with this student customer. It must be formatted like this example:
     :
     :
     :
    """),
    #Definição do agente que irá realizar a tarefa
    agent=agent,
    #Definição da saída esperada da tarefa
    expected_output='A refined finalized version of the marketing campaign in markdown format'
  )

In [ ]:
def get_ad_campaign_written_task(agent, selection):
    return Task(
      #Definição da descrição da tarefa
      description=dedent(f"""\
    You're creating a targeted marketing campaign tailored to what we know about our student customer.

    For each student customer, we have chosen three courses to promote in the next campaign.
    This selection is tailored specifically to the customer: {selection},

    To end this campaign succesfully we will need a promotional message advertising these courses  to the student customer with the ultimate intent that they buy from us.
    This message should be around 3 paragraphs, so that it can be easily integrated into the full letter. For example:
    Interested in learning data science, get yourself enrolled in this course from Harvard University.
    Take Your career to the next level with the help of this course.

    You need to review, approve, and delegate follow up work if necessary to have the complete promotional message. When delegating work send the full draft
  as part of the information.

    Your final answer MUST include the 3 courses from the list, each with a short promotional message.
    """),
    #Definição do agente que irá realizar a tarefa
    agent=agent,
    #Definição da saída esperada da tarefa
    expected_output='A refined finalized version of the marketing campaign in markdown format'
  )

RODANDO O CREW PARA OBTER O RESULTADO FINAL

In [ ]:
# Lista para armazenar os resultados
df_output_list = [] 

for index, row in df_customers.iterrows():
  print('############################################## '+ str(index))
  customer_description = f'''
  Their academic goals are {row['Academic Goals']}.
  Their major is in {row[' Major']}.
  Their Hobbies are {row[' Hobbies']}.
  Their computer skills are {row[' Computer Skills']}.
  Their interest in languages are {row[' Interest in Languages']}.
  Their GPA is {row[' GPA']}.
  '''
  print(customer_description)
  
  #Definição da task 1 para a seleção de cursos
  task1 = get_ad_campaign_task(Chief_Recommendation_Director ,customer_description, courses)
  #Da início a uma equipe de agentes para realizar a tarefa
  targetting_crew = Crew(
    agents=[student_profiler, course_specialist ,Chief_Recommendation_Director ],
    tasks=[task1],
    verbose=True, 
  #Tarefas executadas uma após a outra e o resultado da anterior é passado como conteúdo extra para a próxima.
  process=Process.sequential 
  )
  targetting_result = targetting_crew.kickoff()
  
  #Definição da task 2 para a criação de uma mensagem promocional
  task2 = get_ad_campaign_written_task(Chief_Recommendation_Director ,targetting_result)
  #Da início a uma equipe de agentes para realizar a tarefa
  copywriting_crew = Crew(
    agents=[campaign_agent,Chief_Recommendation_Director ],
    tasks=[task2],
    verbose=True, 
  #Tarefas executadas uma após a outra e o resultado da anterior é passado como conteúdo extra para a próxima.
  process=Process.sequential
  )
  copywriting_result = copywriting_crew.kickoff()

  #Armazena os resultados
  df_output_list.append({'customer':customer_description,
                         'targeted_courses':targetting_result,
                         'promo_msg':copywriting_result,
                        })

#Transforma a lista de resultados em um DataFrame do pandas para visualização
df_output = pd.DataFrame(df_output_list)
print(df_output)

############################################## 0

  Their academic goals are To become a software engineer.
  Their major is in  Computer Science.
  Their Hobbies are  Gaming.
  Their computer skills are  Advanced.
  Their interest in languages are  Spanish.
  Their GPA is 3.7.
  


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: ab994cc1-968f-4603-af36-093dcbbfb319                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 8f75c236-84eb-4237-a5fc-90c22bfad1b8
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 8f75c236-84eb-4237-a5fc-90c22bfad1b8
       Status: Executing Task...
    └── 🤖 Agent: Chief Recomeendation Director
            Status: In Progress

# Agent: Chief Recomeendation Director
## Task:     You're creating a targeted marketing campaign tailored to what we know about our student customers.

    For each student customer, we have to choose exactly three courses to promote in the next campaign.
    Make sure the selection is the best possible and aligned with the student customer,
   review, approve, ask clarifying question or delegate follow up work if
  necessary to make decisions. When delegating work send the full draft
  as part of the information.
    This is the list of all the courses participating in the campaign: 
"Introduction to Computer Science" - Offered by Harvard University on edX
"Biology: Life on Earth" - Offered by Coursera
"Introduction to Psychology" - Offered by Yale University on Coursera
"Environmental Science" - Offered by University of Leeds on FutureLearn
"Introduction to Literature" - Offered by MIT on edX
"Medical Terminology" - Offered by University of Pittsburgh on Coursera
"Data Science and M

...

🚀 Crew: crew
└── 📋 Task: d8e556b0-bd1a-4563-b24d-17f178cff6af
       Status: Executing Task...
    └── 🤖 Agent: Chief Recomeendation Director
            Status: ✅ Completed

🚀 Crew: crew
└── 📋 Task: d8e556b0-bd1a-4563-b24d-17f178cff6af
       Assigned to: Chief Recomeendation Director
       Status: ✅ Completed
    └── 🤖 Agent: Chief Recomeendation Director
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: d8e556b0-bd1a-4563-b24d-17f178cff6af                                                                     │
│  Agent: Chief Recomeendation Director                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: f2ee379d-dbdb-45b1-988e-384f75520a7e                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

                                            customer  \
0  \n  Their academic goals are To become a softw...   
1  \n  Their academic goals are To study environm...   
2  \n  Their academic goals are To pursue a caree...   
3  \n  Their academic goals are To major in psych...   
4  \n  Their academic goals are To work in intern...   
5  \n  Their academic goals are To become a teach...   
6  \n  Their academic goals are To study literatu...   
7  \n  Their academic goals are To pursue a caree...   
8  \n  Their academic goals are To become a biolo...   
9  \n  Their academic goals are To work in data a...   

                                    targeted_courses  \
0  1. "Introduction to Computer Science" - Offere...   
1  1. "Environmental Science" - Offered by Univer...   
2  1. "Medical Terminology" - Offered by Universi...   
3  1. "Introduction to Psychology" - Offered by Y...   
4  Based on the information provided, the three c...   
5  1. "Introduction to Computer Science" - Offe